In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm #for progress bar
from bs4 import BeautifulSoup #for scraping website
from sklearn.feature_extraction.text import TfidfVectorizer #for word embedding
import re
from scipy import sparse
import time 
import optuna #library for optimization
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100

from sklearn.linear_model import Ridge
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold #for cross-validation of data

In [ ]:
#df = pd.read_csv('../input/jigsawcommentclassifier/archive/train.csv')
#df.head()

In [ ]:
#df['y_nb'] = (
   # df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
    #.sum(axis=1) > 0 ).astype(int)

In [ ]:
#df['y_nb'].unique()

In [ ]:
if 1 == 0 :
    toxicity_weight = {'obscene' : 0.02 , 'toxic' : 0.04 , 'threat' : 0.30 ,
                     'severe_toxic' : 0.19 , 'insult' : 0.10 , 'identity_hate' : 0.20 }
    for val in toxicity_weight:
        df[val] = df[val] * toxicity_weight[val]
    df['y_rr'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis = 1) 

In [ ]:
#train_csv = df[['comment_text' , 'y_nb' , 'y_rr']].rename(
        #columns = {'comment_text' : 'text'})

In [ ]:
#train_csv.head(1)

In [ ]:
#df_2 = pd.read_csv('../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv')
#df_2.head(2)

In [ ]:
#train_csv_2 = df_2[['txt' , 'offensiveness_score']].rename(columns = 
                                        #{'txt' : 'text' , 'offensiveness_score' : 'score'})

In [ ]:
#train_csv_2['y_nb'] = (train_csv_2['score'] > 0).astype(int)
#train_csv_2.head(2)

In [ ]:
if 1 == 0:
    train_csv_2['y_rr'] = train_csv_2['score'].copy()
    train_csv_2.loc[train_csv_2['y_rr'] < 0 , 'y_rr'] = 0
    train_csv_2.drop(['score'] , axis = 1 , inplace = True)
    train_csv_2.sample(2)

In [ ]:
if 1==0:
    df3 = pd.read_csv(
        "/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
    print(len(df3))
    df3['y_nb'] = (
        df3[['toxic', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack']] >= .5 ).any(axis=1).astype(int)

In [ ]:
if 1 == 0:
    toxicity_weight = {'obscene' : 0.02 , 'toxic' : 0.04 , 'threat' : 0.30 ,
                 'severe_toxic' : 0.19 , 'insult' : 0.10 , 'identity_hate' : 0.20 }
    for val in toxicity_weight:
        df_3[val] = df_3[val] * toxicity_weight[val]
    df_3['y_rr'] = df_3[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis = 1) 
    df_3.loc[df_3['y_nb']>0,['comment_text','toxic', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack','y_nb']].sample(5)

In [ ]:
if 1==0:
    train_csv_3 = df_3[['comment_text', 'y_nb','y_rr']].rename(
        columns={'comment_text': 'text'})

In [ ]:
if 1==0:
    print(f"length of first trainig {len(train_csv)}")
    print(f"first training has {(train_csv['y_nb'] == 1).sum()}  toxic comment")
    print(f"first training has  {(train_csv['y_nb'] ==0).sum()} non-toxic comment")

    print(f"length of first trainig {len(train_csv_2)}")
    print(f"first training has {(train_csv_2['y_nb'] == 1).sum()}toxic comment")
    print(f"first training has {(train_csv_2['y_nb'] ==0).sum()} non-toxic comment")
    

In [ ]:
#train_csv.head(3)

In [ ]:
#train_csv_2.head(3)

In [ ]:
def text_cleaning(text):
    
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [ ]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

def clean(data, col):
    
    data[col] = data[col].str.replace('https?://\S+|www\.\S+', ' social medium ', regex=True)      
        
    data[col] = data[col].str.lower()
    data[col] = data[col].str.replace("4", "a") 
    data[col] = data[col].str.replace("2", "l")
    data[col] = data[col].str.replace("5", "s") 
    data[col] = data[col].str.replace("1", "i") 
    data[col] = data[col].str.replace("!", "i") 
    data[col] = data[col].str.replace("|", "i", regex=False) 
    data[col] = data[col].str.replace("0", "o") 
    data[col] = data[col].str.replace("l3", "b") 
    data[col] = data[col].str.replace("7", "t") 
    data[col] = data[col].str.replace("7", "+") 
    data[col] = data[col].str.replace("8", "ate") 
    data[col] = data[col].str.replace("3", "e") 
    data[col] = data[col].str.replace("9", "g")
    data[col] = data[col].str.replace("6", "g")
    data[col] = data[col].str.replace("@", "a")
    data[col] = data[col].str.replace("$", "s", regex=False)
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ", regex=False)
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ", regex=False)
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you', regex=True)
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ", regex=False)
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=False)
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ", regex=False)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=False)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=False)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=False)
    data[col] = data[col].str.replace('\s+', ' ', regex=True)  # will remove more than one whitespace character
#     text = re.sub(r'\b([^\W\d_]+)(\s+\1)+\b', r'\1', re.sub(r'\W+', ' ', text).strip(), flags=re.I)  # remove repeating words coming immediately one after another
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1', regex=True) # 2 or more characters are replaced by 2 characters
#     text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    data[col] = data[col].str.replace("[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", '', regex=True)
    
    
    data[col] = data[col].str.replace(r"what's", "what is ")    
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=False)
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ", regex=False)
    data[col] = data[col].str.replace(r"i'm", "i am ", regex=False)
    data[col] = data[col].str.replace(r"\'re", " are ", regex=False)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=False)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=False)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=False)
    data[col] = data[col].str.replace(r"\'s", " ", regex=False)

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', regex=True)
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1', regex=True)    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ', regex=True)    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    tqdm.pandas()
    data[col] = data[col].progress_apply(text_cleaning)
    return data

In [ ]:
#train_csv['text'] = train_csv['text'].apply(text_cleaning)
#train_csv.head(1)

In [ ]:

#train_csv_2['text'] = train_csv_2['text'].apply(text_cleaning)
#train_csv_2.head(1)

In [ ]:
#train = pd.concat([train_csv , train_csv_2])
#len(train)

In [ ]:
#train = clean(train , 'text')

In [ ]:
#train.head()

In [ ]:
validation  = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
submission = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
validation = clean(validation , 'less_toxic')
validation = clean(validation , 'more_toxic')

In [ ]:
validation['less_toxic'] = validation['less_toxic'].apply(text_cleaning)
validation['more_toxic'] = validation['more_toxic'].apply(text_cleaning)

In [ ]:
submission['text'] = submission['text'].apply(text_cleaning)
submission = clean(submission , 'text')

In [ ]:
submission.head(2)

In [ ]:
train = pd.read_csv('../input/jigsaw-regression-based-data/train_data_version2.csv')
train.head()

In [ ]:
vec = TfidfVectorizer(lowercase=True, analyzer='char_wb', ngram_range = (2,5))
X = vec.fit_transform(train['text'])
X_less_toxic = vec.transform(validation['less_toxic'])
X_more_toxic = vec.transform(validation['more_toxic'])
X_test  = vec.transform(submission['text'])

In [ ]:
model_1 = Ridge(alpha=0.5)
model_1.fit(X, train['y'])
print(f'Model 1 validation accuracy score:  {(model_1.predict(X_less_toxic) < model_1.predict(X_more_toxic)).mean()}')

model_2 = Ridge(alpha=5)
model_2.fit(X, train['y'])
print(f'Model 2 validation accuracy score:  {(model_2.predict(X_less_toxic) < model_2.predict(X_more_toxic)).mean()}')


model_3 = Ridge(alpha=50)
model_3.fit(X, train['y'])
print(f'Model 3 validation accuracy score: {(model_3.predict(X_less_toxic) < model_3.predict(X_more_toxic)).mean()}')

In [ ]:
p1 = model_1.predict(X_test)
p2 = model_2.predict(X_test)
p3 = model_3.predict(X_test)

In [ ]:
submission['score'] = (p1 + p2 + p3) / 3

In [ ]:
import scipy
from scipy.stats import rankdata

test = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
test['score'] = rankdata(submission['score'].values, method='ordinal')
test[['comment_id', 'score']].to_csv("submission.csv", index=False)